# 필요한 hdf5파일을 하나의 csv파일로 바꾸는 코드입니다.

In [1]:
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
from dplython import *
from scipy.stats import linregress
from IPython.display import Image
from mpl_toolkits.basemap import Basemap
from matplotlib.colors import Normalize
import matplotlib
import matplotlib.cm as cm
import seaborn as sns
from matplotlib import rcParams
from netCDF4 import Dataset
import struct
import binascii
from mpl_toolkits.basemap import addcyclic
from netCDF4 import num2date, date2num, date2index
import datetime
from pyhdf.SD import SD, SDC
import h5py
import glob
import math

C:\Users\hahw9\Anaconda3\envs\cuda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\hahw9\Anaconda3\envs\cuda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\hahw9\Anaconda3\envs\cuda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\hahw9\Anaconda3\envs\cuda\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
data_path = '/home/data4/temporary/Fire_emission_GFED4/'  # -> 현재 저희 서버상 GFED파일이 저장된 경로입니다.
# GFED자료(사용할 데이터들이 담겨있는 폴더)가 저장되어 있는 경로를 입력

save_file_path = '/home/seohyunwoo/wild_code/wildfire/'
# 결과로 나오는 그래프를 사진(png)으로 저장할 폴더 입력

# 반드시 둘다 마지막 폴더이름 뒤에 '/' 를 붙여주어야합니다

In [3]:
# 잠깐 제 환경에 맞게 경로를 바꿔보았습니다.
data_path = 'C:/Users/hahw9/For Coding/Model/hdf5/Data/'
save_file_path = 'C:/Users/hahw9/For Coding/Model/hdf5/result/'

In [4]:
os.chdir(data_path)
h5py_data_list = glob.glob("GFED4.1s*.hdf5")
# hdf에서와 마찬가지로 glob함수로 hdf5를 불러와줍니다
use_data_list = h5py_data_list[:-3]
# 필요한 데이터를 인덱싱 해줍니다.

In [5]:
month_list = []
for i in range(1,13):
    if len(str(i)) == 1:
        month_list.append("0"+str(i))
    else:
        month_list.append(str(i))
# hdf5에는 12개월치의 데이터가 들어있고 01,02,03 - 12까지로 이루어져있으므로 해당 데이터들을 위해 문자로 이루어진 리스트를 만들었습니다.

In [6]:
print(month_list)

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']


In [7]:
def extract_hdf5_monthly_data(data_path, file_name, str_month):
    # data_path -> 데이터가 있는 경로, file_name -> 뽑을 파일의 이름, str_month -> 데이터를 추출할 달 설정
    f = h5py.File(data_path + file_name, 'r')
    BA = f.get('burned_area')
    monthly_data = BA.get(str_month)
    BF = monthly_data.get('burned_fraction')
    # hdf5에서는 burned_area라는 데이터 속에 12개월로 나누어진 burned_fraction이라는 데이터가 있고 그 데이터를 리턴해줍니다.
    return BF

In [8]:
def change_to_1d(change_data):
    change_data1d = np.reshape(change_data, (1, np.product(change_data.shape)))[0]
    # hdf파일과 마찬가지로 그림을 그리기 위해 1d로 변환하여줍니다.
    return change_data1d

In [9]:
def extract_lon(hdf5_data):
    f = h5py.File(hdf5_data, 'r')
    lon = f.get('lon')
    lon_array = np.array(lon)
    return lon_array

def extract_lat(hdf5_data):
    f = h5py.File(hdf5_data, 'r')
    lat = f.get('lat')
    lat_array = np.array(lat)
    return lat_array
# hdf파일에서와 마찬가지로 위도, 경도를 뽑는 함수입니다.

In [10]:
lon = extract_lon(h5py_data_list[0])
lat = extract_lat(h5py_data_list[0])
# 위도 경도 데이터를 같이 저장하기 위해서 임의의 hdf5 데이터를 불러와서 위도와 경도만 따로 저장

lon_1d = change_to_1d(lon)
# 나중에 저장하기 위해 사용할 1d lon_data
lat_1d = change_to_1d(lat)
# 나중에 저장하기 위해 사용할 1d lat_data

In [11]:
df_data = pd.DataFrame()
df_data['lon'] = lon_1d
df_data['lat'] = lat_1d
# 위에서 만들었던 1d 위도,경도 데이터를 df_data라는 판다스 데이터 프레임에 저장합니다

In [12]:
df_data.head()
# 위에서 만든 판다스 데이터 프레임

,lon,lat
0,-179.875,89.875
1,-179.625,89.875
2,-179.375,89.875
3,-179.125,89.875
4,-178.875,89.875


In [13]:
i = 0
for year in range(16):
    for month in month_list:
        df_data[i] = change_to_1d(extract_hdf5_monthly_data(data_path, use_data_list[year], month))
        # 데이터를 뽑고 1d로 바꾼후 df_data의 'i'라는 이름에 저장을 하게 됩니다.
        # 여기서는 2001년 1월의 burned fraction이 '0' 2월은 '1'로 저장이 되며 마지막 데이터인 2016년 12월 데이터는 '191'에 저장되게됩니다.
        i += 1
    if i % 20 == 0:
        print("진행율 : {:.2f}%".format((i/192)*100))
print("완료")

진행율 : 31.25%
진행율 : 62.50%
진행율 : 93.75%
완료


In [19]:
print(df_data.shape)
df_data.head()

(1036800, 194)


,lon,lat,0,1,2,3,4,5,6,7,...,182,183,184,185,186,187,188,189,190,191
0,-179.875,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-179.625,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-179.375,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-179.125,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-178.875,89.875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
df_data.to_csv(save_file_path + 'GFED4.1_total_data.csv')
print('데이터 저장 완료')

데이터 저장 완료
